# Functional groups in stk

by: Andrew Tarzia

### Update 25/02/24

Installation of py3Dmol using `pip install py3Dmol` is required.

# Imports 

In [1]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

from rdkit import Chem 
from rdkit.Chem import AllChem as rdkit
from collections import defaultdict
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdDistGeom
IPythonConsole.ipython_3d = True

import py3Dmol
from IPython.display import Image
import matplotlib.pyplot as plt
import subprocess
import time
import stk
import stko
import spindry as spd
%matplotlib inline

In [2]:
def show_stk_mol(stk_mol):
    data = rdkit.MolToMolBlock(stk_mol.to_rdkit_mol())
    p = py3Dmol.view(
        data=data,
        style={'stick':{'colorscheme':'cyanCarbon'}}, 
        width=400,
        height=400,
    )
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    p.show()

# Working

In [3]:
bb3 = stk.BuildingBlock(
    smiles='O=CCCBr',
    functional_groups=[stk.AldehydeFactory(), stk.BromoFactory()],
)
print(bb3)
show_stk_mol(bb3)

BuildingBlock('O=CCCBr', (Aldehyde(C(1), O(0), H(5), C(2), bonders=(C(1),), deleters=(O(0),)), Bromo(Br(4), C(3), bonders=(C(3),), deleters=(Br(4),))))


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [4]:
bb = stk.BuildingBlock(
    smiles='BrCCCBr',
    functional_groups=[
        stk.Bromo(
            # The number is the atom's id.
            bromine=stk.Br(0),
            atom=stk.C(1),
            # bonders are atoms which have bonds added during
            # construction.
            bonders=(stk.C(1), ),
            # deleters are atoms which are deleted during
            # construction.
            deleters=(stk.Br(0), ),
        ),
    ],
)

In [5]:
print(bb)

BuildingBlock('BrCCCBr', (Bromo(Br(0), C(1), bonders=(C(1),), deleters=(Br(0),)),))


In [6]:
bb = stk.BuildingBlock(
    smiles='BrCCCBr',
    functional_groups=[
        stk.BromoFactory(),
    ],
)

In [7]:
print(bb)

BuildingBlock('BrCCCBr', (Bromo(Br(0), C(1), bonders=(C(1),), deleters=(Br(0),)), Bromo(Br(4), C(3), bonders=(C(3),), deleters=(Br(4),))))


In [8]:
building_block = stk.BuildingBlock(
    smiles='BrCCCBr',
    functional_groups=(
        stk.SmartsFunctionalGroupFactory(
            smarts='[Br][C]',
            bonders=(1, ),
            deleters=(0, ),
        ),
    ),
)

In [9]:
print(building_block)

BuildingBlock('BrCCCBr', (GenericFunctionalGroup(atoms=(Br(0), C(1)), bonders=(C(1),), deleters=(Br(0),)), GenericFunctionalGroup(atoms=(Br(4), C(3)), bonders=(C(3),), deleters=(Br(4),))))


In [10]:
ca = stk.BuildingBlock(
    smiles='CN1C=NC2=C1C(=O)N(C(=O)N2C)C'
)

In [11]:
show_stk_mol(ca)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [12]:
bb1 = stk.BuildingBlock.init_from_molecule(
    ca, 
    functional_groups=(
        stk.SmartsFunctionalGroupFactory(
            smarts='[#6]=[#8]',
            bonders=(0, ),
            deleters=(1, ),
        ),
    )
)
print(bb1)

bb2 = stk.BuildingBlock.init_from_molecule(
    ca, 
    functional_groups=(
        stk.SmartsFunctionalGroupFactory(
            smarts='[#7][#6H3]([#1])([#1])[#1]',
            bonders=(0, ),
            deleters=(1, 2, 3, 4),
        ),
    )
)
print(bb2)
bb2 = bb2.with_functional_groups(bb2.get_functional_groups(fg_ids=(0, )))
print(bb2)

BuildingBlock('Cn1c(=O)c2c(ncn2C)n(C)c1=O', (GenericFunctionalGroup(atoms=(C(6), O(7)), bonders=(C(6),), deleters=(O(7),)), GenericFunctionalGroup(atoms=(C(9), O(10)), bonders=(C(9),), deleters=(O(10),))))
BuildingBlock('Cn1c(=O)c2c(ncn2C)n(C)c1=O', (GenericFunctionalGroup(atoms=(N(1), C(0), H(14), H(15), H(16)), bonders=(N(1),), deleters=(C(0), H(14), H(15), H(16))), GenericFunctionalGroup(atoms=(N(8), C(13), H(21), H(22), H(23)), bonders=(N(8),), deleters=(C(13), H(21), H(22), H(23))), GenericFunctionalGroup(atoms=(N(11), C(12), H(18), H(19), H(20)), bonders=(N(11),), deleters=(C(12), H(18), H(19), H(20)))))
BuildingBlock('Cn1c(=O)c2c(ncn2C)n(C)c1=O', (GenericFunctionalGroup(atoms=(N(1), C(0), H(14), H(15), H(16)), bonders=(N(1),), deleters=(C(0), H(14), H(15), H(16))), GenericFunctionalGroup(atoms=(N(8), C(13), H(21), H(22), H(23)), bonders=(N(8),), deleters=(C(13), H(21), H(22), H(23))), GenericFunctionalGroup(atoms=(N(11), C(12), H(18), H(19), H(20)), bonders=(N(11),), deleters=(C

In [13]:
fake_molecule = stk.ConstructedMolecule(stk.polymer.Linear(
    building_blocks=(bb1, bb2),
    repeating_unit='AB',
    num_repeating_units=1,
))
show_stk_mol(fake_molecule)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [14]:
show_stk_mol(ca)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol